ANN Model for mobile price classification with hyperparameter tuning using keras tuner

In [1]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.6 MB/s eta 0:00:00


# 1. Import Necessary Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer

#2. Load and Preprocess the  Dataset

In [3]:
Data=pd.read_csv('/content/mobile_price_classification.csv')
Data.head(3)

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2


In [4]:
#check missing values
Data.isnull().sum()

,0
battery_power,0
bluetooth,0
clock_speed,0
dual_sim,0
front_cam,0
4G,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


No missing values

In [5]:
#Check duplicates

Data.duplicated().sum()

np.int64(0)

In [6]:
#Checking data types
print(Data.dtypes)

battery_power       int64
bluetooth           int64
clock_speed       float64
dual_sim            int64
front_cam           int64
4G                  int64
int_memory          int64
m_dep             float64
mobile_wt           int64
n_cores             int64
primary_camera      int64
px_height           int64
px_width            int64
ram                 int64
sc_h                int64
sc_w                int64
talk_time           int64
three_g             int64
touch_screen        int64
wifi                int64
price_range         int64
dtype: object


In [7]:
# X contain all columns except 'price_range'
X = Data.drop('price_range', axis=1).values

# y is target labels
y = Data['price_range'].values

#3.Encode the Labels

In [8]:
# Convert labels to one-hot vectors because they are categorical (0,1,2,3)
# This is required for multi-class classification with ANN
lb = LabelBinarizer()
y = lb.fit_transform(y)

#4.Scale the Features

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


# Split Data

In [10]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


#Build Model with Hyperparameters

In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch


def build_model(hp):
    model = Sequential()

    # Hidden layers (1 to 3)
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Choice(f'units_{i}', [32, 64, 128]),
            activation=hp.Choice('activation', ['relu', 'tanh'])
        ))
        model.add(Dropout(rate=hp.Choice('dropout', [0.0, 0.2, 0.3])))

    # Output layer (4 classes for price_range: 0,1,2,3)
    model.add(Dense(4, activation='softmax'))

    # Optimizer
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    lr_choice = hp.Choice('learning_rate', [0.01, 0.001, 0.0001])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_choice)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_choice)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_choice)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


#Hyperparameter Tuning

In [12]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,          # try 10 different hyperparameter combinations
    directory='mobile_tuning',
    project_name='mobile_price_classification'
)

tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=32)


Trial 10 Complete [00h 00m 16s]
val_accuracy: 0.8025000095367432

Best val_accuracy So Far: 0.9775000214576721
Total elapsed time: 00h 02m 56s


#Get the Best Model

In [13]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(1)[0]

print("Best Hyperparameters:", best_hps.values)


Best Hyperparameters: {'num_layers': 3, 'units_0': 32, 'activation': 'tanh', 'dropout': 0.3, 'optimizer': 'rmsprop', 'learning_rate': 0.001, 'units_1': 64, 'units_2': 128}


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#Train the Best Model

In [14]:
best_model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val))


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8917 - loss: 0.2586 - val_accuracy: 0.9475 - val_loss: 0.1319
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8909 - loss: 0.2724 - val_accuracy: 0.9600 - val_loss: 0.1187
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8835 - loss: 0.2662 - val_accuracy: 0.9525 - val_loss: 0.1277
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8791 - loss: 0.2468 - val_accuracy: 0.9525 - val_loss: 0.1302
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9059 - loss: 0.2620 - val_accuracy: 0.9550 - val_loss: 0.1200
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8949 - loss: 0.2414 - val_accuracy: 0.9575 - val_loss: 0.1175
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8974 - loss: 0.2409 - val_accuracy: 0.9725 - val_loss: 0.1132
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9016 - loss: 0.2316 - val_accuracy: 0.9550 - val_loss:

#Evaluate and Predict

In [15]:
loss, acc = best_model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {acc:.4f}")

# Predict on new/unseen data
# The new_data array should contain all 20 features in the same order as the original training data.
# Replace the example features with actual data for prediction.
new_data = np.array([[1000, 1, 1.5, 0, 5, 1, 32, 0.5, 150, 4, 12, 500, 1000, 2000, 15, 10, 10, 1, 1, 1]])  # example features with 20 columns
new_data_scaled = scaler.transform(new_data)
prediction = best_model.predict(new_data_scaled)

# Convert one-hot back to price_range
predicted_price_range = lb.inverse_transform(prediction)
print("Predicted price range:", predicted_price_range[0])

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9806 - loss: 0.0616 
Validation Accuracy: 0.9750
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Predicted price range: 1


The ANN model achieved a validation accuracy of 97%, indicating it predicts mobile price ranges very effectively.

For the new mobile input, the model predicted price range 1, showing it can generalize to unseen data.

Validation metrics

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
y_val_pred = best_model.predict(x_val)
y_val_pred_classes = lb.inverse_transform(y_val_pred)
print(confusion_matrix(lb.inverse_transform(y_val), y_val_pred_classes))
print(classification_report(lb.inverse_transform(y_val), y_val_pred_classes))


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[[105   0   0   0]
 [  1  90   0   0]
 [  0   4  84   4]
 [  0   0   1 111]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       105
           1       0.96      0.99      0.97        91
           2       0.99      0.91      0.95        92
           3       0.97      0.99      0.98       112

    accuracy                           0.97       400
   macro avg       0.98      0.97      0.97       400
weighted avg       0.98      0.97      0.97       400



Predicting on multiple unseen data

In [17]:
import numpy as np

# Example: 3 new mobiles, each with 20 features
new_mobiles = np.array([
    [1000, 1, 1.5, 0, 5, 1, 32, 0.5, 150, 4, 12, 500, 1000, 2000, 15, 10, 10, 1, 1, 1],
    [800, 0, 1.0, 1, 2, 0, 16, 0.3, 120, 2, 5, 400, 800, 1000, 12, 8, 9, 0, 1, 0],
    [1500, 1, 2.5, 1, 8, 1, 64, 1.0, 180, 8, 20, 600, 1200, 3000, 18, 12, 12, 1, 1, 1]
])

# Scale the new data using the same scaler
new_mobiles_scaled = scaler.transform(new_mobiles)

# Predict probabilities using the trained ANN
predictions = best_model.predict(new_mobiles_scaled)

# Convert one-hot predictions back to price_range labels
predicted_price_ranges = lb.inverse_transform(predictions)

# Display the results
for i, price in enumerate(predicted_price_ranges):
    print(f"Mobile {i+1} predicted price range: {price}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Mobile 1 predicted price range: 1
Mobile 2 predicted price range: 0
Mobile 3 predicted price range: 3


The ANN model accurately predicts the price range of mobile phones based on their features, achieving high validation accuracy. It can also generalize to unseen mobiles, providing reliable price range estimates for new devices.